In [12]:
#standard imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [13]:
#read in and print csv

df = pd.read_csv('ZILLOW_DATA.csv')
print(df.head())
print(df.info())

  indicator_id  region_id        date     value
0         ZATT       3101  1998-01-31  338849.0
1         ZATT       3101  1998-02-28  342993.0
2         ZATT       3101  1998-03-31  346763.0
3         ZATT       3101  1998-04-30  349356.0
4         ZATT       3101  1998-05-31  351981.0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159415321 entries, 0 to 159415320
Data columns (total 4 columns):
 #   Column        Dtype  
---  ------        -----  
 0   indicator_id  object 
 1   region_id     int64  
 2   date          object 
 3   value         float64
dtypes: float64(1), int64(1), object(2)
memory usage: 4.8+ GB
None


In [14]:
#the data is rather large
#getting more information to increase process speeds
print(df.dtypes)
print(df.memory_usage(deep=True))

indicator_id     object
region_id         int64
date             object
value           float64
dtype: object
Index                   128
indicator_id     9724334581
region_id        1275322568
date            10680826507
value            1275322568
dtype: int64


In [15]:
#date and indicator_id taking up most of the memory
#we can convert indicator_id to a category as there are few unique values
#we'll also take this opportunity to convert the date column to a datetime dtype
df2 = df.copy()
df2['indicator_id'] = df2['indicator_id'].astype('category')
df2['date'] = pd.to_datetime(df2['date']).copy()
#check to see dtypes and new memory usage
print(df2.dtypes)
print(df2.memory_usage(deep=True))

indicator_id          category
region_id                int64
date            datetime64[ns]
value                  float64
dtype: object
Index                  128
indicator_id     159420841
region_id       1275322568
date            1275322568
value           1275322568
dtype: int64


In [16]:
#the memory usage is drastically reduced
#in final version of this code, we will read the csv with these dtypes
#we can also change region_id to a category dtype and value to int as there are no decmials
df2['region_id'] = df2['region_id'].astype('category')
print(df2.dtypes)
print(df2.memory_usage(deep=True))

indicator_id          category
region_id             category
date            datetime64[ns]
value                  float64
dtype: object
Index                  128
indicator_id     159420841
region_id        640489300
date            1275322568
value           1275322568
dtype: int64


In [17]:
#now, we have indicator_id and region_id, but what do they mean?
#two separate csvs for this
indicators = pd.read_csv('ZILLOW_INDICATORS.csv', dtype='category')
print(indicators.head())
print(indicators.info())
regions = pd.read_csv('ZILLOW_REGIONS.csv', dtype='category')
print(regions.head())
print(regions.info())



  indicator_id                                          indicator  \
0         SAAW  Median Sale Price (Smooth & Seasonally Adjuste...   
1         SRAM        Median Sale Price (Raw, All Homes, Monthly)   
2         NSAM  Median Days to Pending (Smooth, All Homes, Mon...   
3         RSNA  ZORI (Smoothed): All Homes Plus Multifamily Ti...   
4         RSSA  ZORI (Smoothed, Seasonally Adjusted): All Home...   

              category  
0  Inventory and sales  
1  Inventory and sales  
2  Inventory and sales  
3              Rentals  
4              Rentals  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   indicator_id  56 non-null     category
 1   indicator     56 non-null     category
 2   category      56 non-null     category
dtypes: category(3)
memory usage: 5.4 KB
None
  region_id region_type                                            

In [18]:

# first, we need to set an index for the new csvs
indicators.set_index('indicator_id', inplace=True)
regions.set_index('region_id', inplace=True)
df2.set_index(['indicator_id','region_id'],inplace=True)


In [19]:
# join csvs and do some exploratory
full_data = df2.join([indicators,regions])
print(full_data.head())
print(full_data['category'].unique)
#separate datasets by category, .copy() to avoid possible view as we will likely modify these datesets from here
print(full_data.head())
rentals = full_data[full_data['category'] == 'Rentals'].copy()
print(rentals.head())
home_values = full_data[full_data['category'] == 'Home values'].copy()
print(home_values.head())
inv_sales = full_data[full_data['category'] == 'Inventory and sales'].copy()
print(inv_sales.head())

                             date     value  \
indicator_id region_id                        
ZATT         3101      1998-01-31  338849.0   
             3101      1998-02-28  342993.0   
             3101      1998-03-31  346763.0   
             3101      1998-04-30  349356.0   
             3101      1998-05-31  351981.0   

                                                       indicator     category  \
indicator_id region_id                                                          
ZATT         3101       ZHVI All Homes- Top Tier Time Series ($)  Home values   
             3101       ZHVI All Homes- Top Tier Time Series ($)  Home values   
             3101       ZHVI All Homes- Top Tier Time Series ($)  Home values   
             3101       ZHVI All Homes- Top Tier Time Series ($)  Home values   
             3101       ZHVI All Homes- Top Tier Time Series ($)  Home values   

                       region_type region  
indicator_id region_id                     
ZATT         3

In [20]:
#for some reason here these dataframes have NaN values, while in the separate zillow_data.py the values were filled
#based on the diff in the code, the most likely cause for this is the modification of the df object after it has been read in, as in the .py this was done during the read process
